In [ ]:
import yaml
with open('prompts.yaml','r') as stream:
    prompt_templates=yaml.safe_load(stream)


In [ ]:
# 使用rag
from langchain.vectorstores import FAISS
from tools.Retriever_tool import RetrieverTool
from langchain.embeddings import HuggingFaceEmbeddings
vectordb = FAISS.load_local(
    "./vectordb",
    embeddings=HuggingFaceEmbeddings(model_name="thenlper/gte-small"),
    allow_dangerous_deserialization=True,
)
retrievertool = RetrieverTool(vectordb=vectordb)

In [ ]:
#知识检索 agent
from smolagents import CodeAgent,OpenAIServerModel,tool,HfApiModel,DuckDuckGoSearchTool
from tools.final_answer import FinalAnswerTool
from tools.visit_webpage import VisitWebpageTool
from tools.vision_comprehension import VisionComprehension
from tools.file_io import file_writer
model=OpenAIServerModel(model_id='qwen-max',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
qwen_agent=CodeAgent(model=model,
                     tools=[DuckDuckGoSearchTool(),file_writer(),retrievertool,VisitWebpageTool()],
                     additional_authorized_imports=['flask','os','matplotlib','pandas','numpy','seaborn','sklearn','torch','transformers','tensorflow','keras','cv2','PIL','matplotlib.pyplot','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL','matplotlib.pyplot as plt','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL'],
                     prompt_templates=prompt_templates,
                     name='qwen_agent',
                     description='information extraction agent,can do rag and web search')


In [ ]:
#管理agent

from smolagents import CodeAgent,OpenAIServerModel,tool,HfApiModel,DuckDuckGoSearchTool
model=OpenAIServerModel(model_id='deepseek-v3',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", prtogether")
manage_agent=CodeAgent(model=model,
                       tools=[DuckDuckGoSearchTool(),file_writer(),FinalAnswerTool(),retrievertool],
                       additional_authorized_imports=['flask','os','matplotlib','pandas','numpy','seaborn','sklearn','torch','transformers','tensorflow','keras','cv2','PIL','matplotlib.pyplot','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL','matplotlib.pyplot as plt','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL'],
                       prompt_templates=prompt_templates,
                       max_steps=15,
                       planning_interval=5,
                       managed_agents=[qwen_agent]
                       )

In [ ]:
manage_agent.visualize()

In [ ]:
import os
import  base64
# Get keys for your project from the project settings page: https://cloud.langfuse.com
LANGFUSE_PUBLIC_KEY =  "pk-lf-1b85b139-ccaf-4cd4-a9e2-ec34c71a1746" 
LANGFUSE_SECRET_KEY = "sk-lf-73fe4c78-d03c-413f-98f8-471424af62b6" 
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-1b85b139-ccaf-4cd4-a9e2-ec34c71a1746" 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-73fe4c78-d03c-413f-98f8-471424af62b6"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region
from langfuse.callback import CallbackHandler
LANGFUSE_AUTH = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [ ]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
 
# Create a TracerProvider for OpenTelemetry
trace_provider = TracerProvider()

# Add a SimpleSpanProcessor with the OTLPSpanExporter to send traces
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# Set the global default tracer provider
from opentelemetry import trace
trace.set_tracer_provider(trace_provider)
tracer = trace.get_tracer(__name__)

# Instrument smolagents with the configured provider
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
manage_agent.run(
"'./data/iris.csv' 是一个鸢尾花数据集，请你帮我完成对它的数据分析任务，请牢记，一定要先规划分析流程，即先进行planning，然后按照分析流程来完成分析任务，所有结果包括图表保存在./workspace/result1目录下    "
, )

In [10]:
ms=manage_agent.write_memory_to_messages(summary_mode=True)

16

In [ ]:
ms[1]

In [ ]:
for i in ms:
    print(i['role'])

In [13]:
from pymongo import MongoClient
client=MongoClient("mongodb://localhost:27017/")
db=client["test"]

In [15]:
db.agent.insert_many(ms)

InsertManyResult([ObjectId('67e4207c8cd9eeee2f46078b'), ObjectId('67e4207c8cd9eeee2f46078c'), ObjectId('67e4207c8cd9eeee2f46078d'), ObjectId('67e4207c8cd9eeee2f46078e'), ObjectId('67e4207c8cd9eeee2f46078f'), ObjectId('67e4207c8cd9eeee2f460790'), ObjectId('67e4207c8cd9eeee2f460791'), ObjectId('67e4207c8cd9eeee2f460792'), ObjectId('67e4207c8cd9eeee2f460793'), ObjectId('67e4207c8cd9eeee2f460794'), ObjectId('67e4207c8cd9eeee2f460795'), ObjectId('67e4207c8cd9eeee2f460796'), ObjectId('67e4207c8cd9eeee2f460797'), ObjectId('67e4207c8cd9eeee2f460798'), ObjectId('67e4207c8cd9eeee2f460799'), ObjectId('67e4207c8cd9eeee2f46079a')], acknowledged=True)

In [17]:
cursor=db.agent.find()

In [ ]:
len(ms)

In [ ]:
temp=[]
for step in manage_agent.memory.steps:
    temp.append(step.to_messages(summary_mode=True))

In [ ]:
len(temp)

In [ ]:
for e in temp:
    print(len(e))

In [ ]:
from smolagents import ActionStep,PlanningStep,TaskStep
messages_step=[]
for i,step in enumerate(manage_agent.memory.steps):
    record={
    }
    if isinstance(step,ActionStep):
        print('ActionStep',i)
        record['instance']='ActionStep'
        record['content']=step.dict()
    elif isinstance(step,PlanningStep):
        print('PlanningStep',i)
        record['instance']='PlanningStep'
        record['content']=step.dict()
    elif isinstance(step,TaskStep):
        print('TaskStep',i)
        record['instance']='TaskStep'
        record['content']=step.dict()
    else:
        print(step)
    messages_step.append(record)
    

In [ ]:
db.drop_collection('myCollection')

In [ ]:
db.create_collection('agent')

In [ ]:
db['agent'].find()

In [ ]:
manage_agent.memory